In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
pip install urduhack

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.5/105.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.8/82.8 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 6.9 MB/s eta 0:00:0000:01:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 42.1 MB/s eta 0:00:00:00:01
  Attempting uninstall: typeguard
    Found existing installation: typeguard 4.1.5
    Uninstalling typeguard-4.1.5:
      Successfully uninstalled typeguard-4.1.5
  Attempting uninstall: keras
    Found existing installation: keras 3.2.1
    Uninstalling keras-3.2.1:
      Successfully uninstalled keras-3.2.1
  Attempting uninstall: Click
    Found existing installation: click 8.1.7
    Uninstalling click-8.1.7:
      Successfully uninstalled click-8.1.7
  Attempting uninstall: tensorflow-datasets
    Found existing installation: tensorflow-datasets 4.9.4
    Uninstalling tensorflow

In [1]:
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
from urduhack import normalize
from urduhack.preprocessing import replace_currency_symbols, replace_numbers, remove_accents, normalize_whitespace 
from urduhack.preprocessing import replace_phone_numbers, replace_emails, replace_urls, remove_punctuation
from urduhack.tokenization import word_tokenizer

/Users/zeeshan/Multilingual-Sentiment-Analysis/venv/lib/python3.8/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [2]:
df = pd.read_csv('bbc_headlines.csv', engine='python')

FileNotFoundError: [Errno 2] No such file or directory: 'bbc_headlines.csv'

In [5]:
df

,Page,Headlineno.,Article,Topic,Source
0,1,1,کیا بولڈو نامی پودے کے پتّے مستقبل میں ٹوائلٹ ...,Science,BBC Urdu
1,1,1,دنیا کے دیگر خطوں کی طرح افریقہ میں بھی ٹوائلٹ...,Science,BBC Urdu
2,1,1,ٹوائلٹ پیپر بنایا تو افریقہ میں بھی جاتا ہے لی...,Science,BBC Urdu
3,1,1,ماہرِ نباتات مارٹن کہتے ہیں کہ ٹوائلٹ پیپر کی ...,Science,BBC Urdu
4,1,1,ان کے مطابق ’آج کل کے نوجوان اس پودے کے بارے م...,Science,BBC Urdu
...,...,...,...,...,...
234758,40,24,کم جان اُن شمالی کوریا کو کبھی بھی آسانی سے مع...,World,BBC Urdu
234759,40,24,پنے آخری بین الاقوامی دوروں کے لیے، فروری 2019...,World,BBC Urdu
234760,40,24,یہ واضح نہیں ہے کہ دونوں رہنماؤں نے اپنی ملاقا...,World,BBC Urdu
234761,40,24,یوکرین پر روس کے حملے کے بعد سے امریکی حکمت عم...,World,BBC Urdu


In [3]:
def preprocess_urdu_text(text):
    if isinstance(text, str):
        text = replace_currency_symbols(text)
        text = replace_numbers(text)
        text = remove_accents(text)
        text = normalize_whitespace(text)
        text = replace_phone_numbers(text)
        text = replace_emails(text)
        text = replace_urls(text)
        text = remove_punctuation(text)
        text = normalize(text)
        return text
    else:
        return ""

In [7]:
df['Preprocessed'] = df['Article'].apply(preprocess_urdu_text)
df = df[df['Preprocessed'].str.strip() != '']

In [8]:
df

,Page,Headlineno.,Article,Topic,Source,Preprocessed
0,1,1,کیا بولڈو نامی پودے کے پتّے مستقبل میں ٹوائلٹ ...,Science,BBC Urdu,کیا بولڈو نامی پودے کے پتے مستقبل میں ٹوائلٹ پ...
1,1,1,دنیا کے دیگر خطوں کی طرح افریقہ میں بھی ٹوائلٹ...,Science,BBC Urdu,دنیا کے دیگر خطوں کی طرح افریقہ میں بھی ٹوائلٹ...
2,1,1,ٹوائلٹ پیپر بنایا تو افریقہ میں بھی جاتا ہے لی...,Science,BBC Urdu,ٹوائلٹ پیپر بنایا تو افریقہ میں بھی جاتا ہے لی...
3,1,1,ماہرِ نباتات مارٹن کہتے ہیں کہ ٹوائلٹ پیپر کی ...,Science,BBC Urdu,ماہر نباتات مارٹن کہتے ہیں کہ ٹوائلٹ پیپر کی ب...
4,1,1,ان کے مطابق ’آج کل کے نوجوان اس پودے کے بارے م...,Science,BBC Urdu,ان کے مطابق آج کل کے نوجوان اس پودے کے بارے می...
...,...,...,...,...,...,...
234758,40,24,کم جان اُن شمالی کوریا کو کبھی بھی آسانی سے مع...,World,BBC Urdu,کم جان ان شمالی کوریا کو کبھی بھی آسانی سے معا...
234759,40,24,پنے آخری بین الاقوامی دوروں کے لیے، فروری 2019...,World,BBC Urdu,پنے آخری بین الاقوامی دوروں کے لیے فروری میں ڈ...
234760,40,24,یہ واضح نہیں ہے کہ دونوں رہنماؤں نے اپنی ملاقا...,World,BBC Urdu,یہ واضح نہیں ہے کہ دونوں رہنماؤں نے اپنی ملاقا...
234761,40,24,یوکرین پر روس کے حملے کے بعد سے امریکی حکمت عم...,World,BBC Urdu,یوکرین پر روس کے حملے کے بعد سے امریکی حکمت عم...


In [9]:
from nltk.tokenize import word_tokenize
tokenized = []
for review in df['Preprocessed']:
    tokens = word_tokenize(review)

    tokenized.append(tokens)

In [10]:
len(tokenized)

231764

In [11]:
df['Tokenized_data'] = tokenized

/var/folders/46/6_4h40v120z9n6mcms1h3yhm0000gp/T/ipykernel_40544/169671364.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Tokenized_data'] = tokenized


In [12]:
df

,Page,Headlineno.,Article,Topic,Source,Preprocessed,Tokenized_data
0,1,1,کیا بولڈو نامی پودے کے پتّے مستقبل میں ٹوائلٹ ...,Science,BBC Urdu,کیا بولڈو نامی پودے کے پتے مستقبل میں ٹوائلٹ پ...,"[کیا, بولڈو, نامی, پودے, کے, پتے, مستقبل, میں,..."
1,1,1,دنیا کے دیگر خطوں کی طرح افریقہ میں بھی ٹوائلٹ...,Science,BBC Urdu,دنیا کے دیگر خطوں کی طرح افریقہ میں بھی ٹوائلٹ...,"[دنیا, کے, دیگر, خطوں, کی, طرح, افریقہ, میں, ب..."
2,1,1,ٹوائلٹ پیپر بنایا تو افریقہ میں بھی جاتا ہے لی...,Science,BBC Urdu,ٹوائلٹ پیپر بنایا تو افریقہ میں بھی جاتا ہے لی...,"[ٹوائلٹ, پیپر, بنایا, تو, افریقہ, میں, بھی, جا..."
3,1,1,ماہرِ نباتات مارٹن کہتے ہیں کہ ٹوائلٹ پیپر کی ...,Science,BBC Urdu,ماہر نباتات مارٹن کہتے ہیں کہ ٹوائلٹ پیپر کی ب...,"[ماہر, نباتات, مارٹن, کہتے, ہیں, کہ, ٹوائلٹ, پ..."
4,1,1,ان کے مطابق ’آج کل کے نوجوان اس پودے کے بارے م...,Science,BBC Urdu,ان کے مطابق آج کل کے نوجوان اس پودے کے بارے می...,"[ان, کے, مطابق, آج, کل, کے, نوجوان, اس, پودے, ..."
...,...,...,...,...,...,...,...
234758,40,24,کم جان اُن شمالی کوریا کو کبھی بھی آسانی سے مع...,World,BBC Urdu,کم جان ان شمالی کوریا کو کبھی بھی آسانی سے معا...,"[کم, جان, ان, شمالی, کوریا, کو, کبھی, بھی, آسا..."
234759,40,24,پنے آخری بین الاقوامی دوروں کے لیے، فروری 2019...,World,BBC Urdu,پنے آخری بین الاقوامی دوروں کے لیے فروری میں ڈ...,"[پنے, آخری, بین, الاقوامی, دوروں, کے, لیے, فرو..."
234760,40,24,یہ واضح نہیں ہے کہ دونوں رہنماؤں نے اپنی ملاقا...,World,BBC Urdu,یہ واضح نہیں ہے کہ دونوں رہنماؤں نے اپنی ملاقا...,"[یہ, واضح, نہیں, ہے, کہ, دونوں, رہنماؤں, نے, ا..."
234761,40,24,یوکرین پر روس کے حملے کے بعد سے امریکی حکمت عم...,World,BBC Urdu,یوکرین پر روس کے حملے کے بعد سے امریکی حکمت عم...,"[یوکرین, پر, روس, کے, حملے, کے, بعد, سے, امریک..."


In [13]:
model = Word2Vec(sentences=tokenized, vector_size=100, window=5, min_count=1)


In [14]:
model.save("urdu_article_word2vec.bin")

In [16]:
print(model)

Word2Vec<vocab=65206, vector_size=100, alpha=0.025>


In [19]:
import os
os.chdir(r'/kaggle/working')

In [20]:
%cd /kaggle/working

/kaggle/working


In [22]:
from IPython.display import FileLink 
FileLink(r'urdu_headline_word2vec.bin')

/kaggle/working/urdu_headline_word2vec.bin

In [4]:
df1 = pd.read_csv('Data/headlines.csv', engine='python')
df2 = pd.read_csv('Data/urdu_tokenized.csv')
df3 = pd.read_csv('Data/bbc-urdu/articles/bbc_headlines.csv', engine='python')

In [5]:
df1

,Headline,Topic,Source
0,"ویڈیو, نوکری بمقابلہ کاروبار: کون زیادہ فائدہ ...",Video,BBC Urdu
1,"ویڈیو, جب لندن کی سڑکوں پر فوجی گھوڑے سوار گِر...",Video,BBC Urdu
2,"ویڈیو, اسرائیلی فضائی حملے میں حاملہ خاتون کی ...",Video,BBC Urdu
3,"ویڈیو, آٹزم سے متاثرہ بچوں کے والدین کی زندگیا...",Video,BBC Urdu
4,"ویڈیو, پاکستان میں ’گھس کر مارنا‘ انڈیا کی پال...",Video,BBC Urdu
...,...,...,...
7436,سونے کی قیمت میں 7 ہزار 800 روپے کی بڑی کمی,Latest 2,Dawn News
7437,ارجنٹائن کا انٹرپول سے ایرانی وزیر داخلہ کو گر...,Latest 2,Dawn News
7438,اپنی بہن کا رشتہ بابر اعظم کو دیں، نازش جہانگی...,Latest 2,Dawn News
7439,بدین: 2 نوعمر لڑکوں کا گینگ ریپ، پولیس اہلکار ...,Latest 2,Dawn News


In [6]:
df2

,Urdu,Preprocessed_Urdu,Tokenized_data
0,ہم نے اک عمر بسر کی ہے غم یار کے ساتھ,ہم نے اک عمر بسر کی ہے غم یار کے ساتھ,"['ہم', 'نے', 'اک', 'عمر', 'بسر', 'کی', 'ہے', '..."
1,اب تو ہم گھر سے نکلتے ہیں تو رکھ دیتے ہیں,اب تو ہم گھر سے نکلتے ہیں تو رکھ دیتے ہیں,"['اب', 'تو', 'ہم', 'گھر', 'سے', 'نکلتے', 'ہیں'..."
2,اس قدر خوف ہے اب شہر کی گلیوں میں کہ لوگ,اس قدر خوف ہے اب شہر کی گلیوں میں کہ لوگ,"['اس', 'قدر', 'خوف', 'ہے', 'اب', 'شہر', 'کی', ..."
3,ایک تو خواب لیے پھرتے ہو گلیوں گلیوں,ایک تو خواب لیے پھرتے ہو گلیوں گلیوں,"['ایک', 'تو', 'خواب', 'لیے', 'پھرتے', 'ہو', 'گ..."
4,شہر کا شہر ہی ناصح ہو تو کیا کیجئے گا,شہر کا شہر ہی ناصح ہو تو کیا کیجئے گا,"['شہر', 'کا', 'شہر', 'ہی', 'ناصح', 'ہو', 'تو',..."
...,...,...,...
254684,288 پوائنٹس ریکور انڈیکس 42004 پر گیا 67 فیصد ...,پوائنٹس ریکور انڈیکس پر گیا فیصد شیئرز کے دام ...,"['پوائنٹس', 'ریکور', 'انڈیکس', 'پر', 'گیا', 'ف..."
254685,132 روپے 56 پیسے پر گیا اوپن مارکیٹ میں 50 پیس...,روپے پیسے پر گیا اوپن مارکیٹ میں پیسے کمی سے ر...,"['روپے', 'پیسے', 'پر', 'گیا', 'اوپن', 'مارکیٹ'..."
254686,اسٹور اور فروخت کرنے پر پابندی ہو گی ڈی جی فوڈ...,اسٹور اور فروخت کرنے پر پابندی ہو گی ڈی جی فوڈ...,"['اسٹور', 'اور', 'فروخت', 'کرنے', 'پر', 'پابند..."
254687,ٹرانسپورٹرز نے کرائے بڑھا دیے لنڈا بازار میں ب...,ٹرانسپورٹرز نے کرائے بڑھا دیے لنڈا بازار میں ب...,"['ٹرانسپورٹرز', 'نے', 'کرائے', 'بڑھا', 'دیے', ..."


In [7]:
df3

,Page,Headlineno.,Article,Topic,Source
0,1,1,کیا بولڈو نامی پودے کے پتّے مستقبل میں ٹوائلٹ ...,Science,BBC Urdu
1,1,1,دنیا کے دیگر خطوں کی طرح افریقہ میں بھی ٹوائلٹ...,Science,BBC Urdu
2,1,1,ٹوائلٹ پیپر بنایا تو افریقہ میں بھی جاتا ہے لی...,Science,BBC Urdu
3,1,1,ماہرِ نباتات مارٹن کہتے ہیں کہ ٹوائلٹ پیپر کی ...,Science,BBC Urdu
4,1,1,ان کے مطابق ’آج کل کے نوجوان اس پودے کے بارے م...,Science,BBC Urdu
...,...,...,...,...,...
234758,40,24,کم جان اُن شمالی کوریا کو کبھی بھی آسانی سے مع...,World,BBC Urdu
234759,40,24,پنے آخری بین الاقوامی دوروں کے لیے، فروری 2019...,World,BBC Urdu
234760,40,24,یہ واضح نہیں ہے کہ دونوں رہنماؤں نے اپنی ملاقا...,World,BBC Urdu
234761,40,24,یوکرین پر روس کے حملے کے بعد سے امریکی حکمت عم...,World,BBC Urdu


In [8]:
df1['Preprocessed'] = df1['Headline'].apply(preprocess_urdu_text)
df1 = df1[df1['Preprocessed'].str.strip() != '']

In [9]:
df2['Preprocessed'] = df2['Urdu'].apply(preprocess_urdu_text)
df2 = df2[df2['Preprocessed'].str.strip() != '']

In [10]:
df3['Preprocessed'] = df3['Article'].apply(preprocess_urdu_text)
df3 = df3[df3['Preprocessed'].str.strip() != '']

In [11]:
df1

,Headline,Topic,Source,Preprocessed
0,"ویڈیو, نوکری بمقابلہ کاروبار: کون زیادہ فائدہ ...",Video,BBC Urdu,ویڈیو نوکری بمقابلہ کاروبار کون زیادہ فائدہ من...
1,"ویڈیو, جب لندن کی سڑکوں پر فوجی گھوڑے سوار گِر...",Video,BBC Urdu,ویڈیو جب لندن کی سڑکوں پر فوجی گھوڑے سوار گرا ...
2,"ویڈیو, اسرائیلی فضائی حملے میں حاملہ خاتون کی ...",Video,BBC Urdu,ویڈیو اسرائیلی فضائی حملے میں حاملہ خاتون کی ہ...
3,"ویڈیو, آٹزم سے متاثرہ بچوں کے والدین کی زندگیا...",Video,BBC Urdu,ویڈیو آٹزم سے متاثرہ بچوں کے والدین کی زندگیاں...
4,"ویڈیو, پاکستان میں ’گھس کر مارنا‘ انڈیا کی پال...",Video,BBC Urdu,ویڈیو پاکستان میں گھس کر مارنا انڈیا کی پالیسی...
...,...,...,...,...
7436,سونے کی قیمت میں 7 ہزار 800 روپے کی بڑی کمی,Latest 2,Dawn News,سونے کی قیمت میں ہزار روپے کی بڑی کمی
7437,ارجنٹائن کا انٹرپول سے ایرانی وزیر داخلہ کو گر...,Latest 2,Dawn News,ارجنٹائن کا انٹرپول سے ایرانی وزیر داخلہ کو گر...
7438,اپنی بہن کا رشتہ بابر اعظم کو دیں، نازش جہانگی...,Latest 2,Dawn News,اپنی بہن کا رشتہ بابر اعظم کو دیں نازش جہانگیر...
7439,بدین: 2 نوعمر لڑکوں کا گینگ ریپ، پولیس اہلکار ...,Latest 2,Dawn News,بدین نوعمر لڑکوں کا گینگ ریپ پولیس اہلکار اور ...


In [12]:
df2

,Urdu,Preprocessed_Urdu,Tokenized_data,Preprocessed
0,ہم نے اک عمر بسر کی ہے غم یار کے ساتھ,ہم نے اک عمر بسر کی ہے غم یار کے ساتھ,"['ہم', 'نے', 'اک', 'عمر', 'بسر', 'کی', 'ہے', '...",ہم نے اک عمر بسر کی ہے غم یار کے ساتھ
1,اب تو ہم گھر سے نکلتے ہیں تو رکھ دیتے ہیں,اب تو ہم گھر سے نکلتے ہیں تو رکھ دیتے ہیں,"['اب', 'تو', 'ہم', 'گھر', 'سے', 'نکلتے', 'ہیں'...",اب تو ہم گھر سے نکلتے ہیں تو رکھ دیتے ہیں
2,اس قدر خوف ہے اب شہر کی گلیوں میں کہ لوگ,اس قدر خوف ہے اب شہر کی گلیوں میں کہ لوگ,"['اس', 'قدر', 'خوف', 'ہے', 'اب', 'شہر', 'کی', ...",اس قدر خوف ہے اب شہر کی گلیوں میں کہ لوگ
3,ایک تو خواب لیے پھرتے ہو گلیوں گلیوں,ایک تو خواب لیے پھرتے ہو گلیوں گلیوں,"['ایک', 'تو', 'خواب', 'لیے', 'پھرتے', 'ہو', 'گ...",ایک تو خواب لیے پھرتے ہو گلیوں گلیوں
4,شہر کا شہر ہی ناصح ہو تو کیا کیجئے گا,شہر کا شہر ہی ناصح ہو تو کیا کیجئے گا,"['شہر', 'کا', 'شہر', 'ہی', 'ناصح', 'ہو', 'تو',...",شہر کا شہر ہی ناصح ہو تو کیا کیجئے گا
...,...,...,...,...
254684,288 پوائنٹس ریکور انڈیکس 42004 پر گیا 67 فیصد ...,پوائنٹس ریکور انڈیکس پر گیا فیصد شیئرز کے دام ...,"['پوائنٹس', 'ریکور', 'انڈیکس', 'پر', 'گیا', 'ف...",پوائنٹس ریکور انڈیکس پر گیا فیصد شیئرز کے دام ...
254685,132 روپے 56 پیسے پر گیا اوپن مارکیٹ میں 50 پیس...,روپے پیسے پر گیا اوپن مارکیٹ میں پیسے کمی سے ر...,"['روپے', 'پیسے', 'پر', 'گیا', 'اوپن', 'مارکیٹ'...",روپے پیسے پر گیا اوپن مارکیٹ میں پیسے کمی سے ر...
254686,اسٹور اور فروخت کرنے پر پابندی ہو گی ڈی جی فوڈ...,اسٹور اور فروخت کرنے پر پابندی ہو گی ڈی جی فوڈ...,"['اسٹور', 'اور', 'فروخت', 'کرنے', 'پر', 'پابند...",اسٹور اور فروخت کرنے پر پابندی ہو گی ڈی جی فوڈ...
254687,ٹرانسپورٹرز نے کرائے بڑھا دیے لنڈا بازار میں ب...,ٹرانسپورٹرز نے کرائے بڑھا دیے لنڈا بازار میں ب...,"['ٹرانسپورٹرز', 'نے', 'کرائے', 'بڑھا', 'دیے', ...",ٹرانسپورٹرز نے کرائے بڑھا دیے لنڈا بازار میں ب...


In [13]:
df3

,Page,Headlineno.,Article,Topic,Source,Preprocessed
0,1,1,کیا بولڈو نامی پودے کے پتّے مستقبل میں ٹوائلٹ ...,Science,BBC Urdu,کیا بولڈو نامی پودے کے پتے مستقبل میں ٹوائلٹ پ...
1,1,1,دنیا کے دیگر خطوں کی طرح افریقہ میں بھی ٹوائلٹ...,Science,BBC Urdu,دنیا کے دیگر خطوں کی طرح افریقہ میں بھی ٹوائلٹ...
2,1,1,ٹوائلٹ پیپر بنایا تو افریقہ میں بھی جاتا ہے لی...,Science,BBC Urdu,ٹوائلٹ پیپر بنایا تو افریقہ میں بھی جاتا ہے لی...
3,1,1,ماہرِ نباتات مارٹن کہتے ہیں کہ ٹوائلٹ پیپر کی ...,Science,BBC Urdu,ماہر نباتات مارٹن کہتے ہیں کہ ٹوائلٹ پیپر کی ب...
4,1,1,ان کے مطابق ’آج کل کے نوجوان اس پودے کے بارے م...,Science,BBC Urdu,ان کے مطابق آج کل کے نوجوان اس پودے کے بارے می...
...,...,...,...,...,...,...
234758,40,24,کم جان اُن شمالی کوریا کو کبھی بھی آسانی سے مع...,World,BBC Urdu,کم جان ان شمالی کوریا کو کبھی بھی آسانی سے معا...
234759,40,24,پنے آخری بین الاقوامی دوروں کے لیے، فروری 2019...,World,BBC Urdu,پنے آخری بین الاقوامی دوروں کے لیے فروری میں ڈ...
234760,40,24,یہ واضح نہیں ہے کہ دونوں رہنماؤں نے اپنی ملاقا...,World,BBC Urdu,یہ واضح نہیں ہے کہ دونوں رہنماؤں نے اپنی ملاقا...
234761,40,24,یوکرین پر روس کے حملے کے بعد سے امریکی حکمت عم...,World,BBC Urdu,یوکرین پر روس کے حملے کے بعد سے امریکی حکمت عم...


In [14]:
df1_preprocessed = df1['Preprocessed']
df2_preprocessed = df2['Preprocessed']
df3_preprocessed = df3['Preprocessed']

In [15]:
df_combined = pd.concat([df1_preprocessed, df2_preprocessed, df3_preprocessed], ignore_index=True)

In [24]:
from nltk.tokenize import word_tokenize
tokenized = []
for review in df1['Preprocessed']:
    tokens = word_tokenize(review)

    tokenized.append(tokens)

In [25]:
print(len(sum(tokenized, [])))

118605


In [26]:
l2 = []
for review in df2['Preprocessed']:
    tokens = word_tokenize(review)
    l2.append(tokens)
    tokenized.append(tokens)

In [27]:
total_words = 0

for tokens in l2:
    total_words += len(tokens)

print("Total number of words:", total_words)


Total number of words: 30770136


In [28]:
total_words = 0

for tokens in tokenized:
    total_words += len(tokens)

print("Total number of words:", total_words)


Total number of words: 30888741


In [29]:
l3 = []
for review in df3['Preprocessed']:
    tokens = word_tokenize(review)
    l3.append(tokens)
    tokenized.append(tokens)

In [30]:
total_words = 0

for tokens in l3:
    total_words += len(tokens)

print("Total number of words:", total_words)


Total number of words: 8027197


In [31]:
total_words = 0

for tokens in tokenized:
    total_words += len(tokens)

print("Total number of words:", total_words)

Total number of words: 38915938


In [23]:
len(tokenized)

493894

In [19]:
model = Word2Vec(sentences=tokenized, vector_size=100, window=5, min_count=1)

In [20]:
model.save("urdu_all_word2vec.bin")

In [21]:
print(model)

Word2Vec<vocab=433720, vector_size=100, alpha=0.025>
